In [26]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=10000, noise=0.4)

In [27]:
X

array([[ 1.57671338,  0.01872579],
       [-0.05186541, -0.20028459],
       [ 1.41306156, -0.57459177],
       ...,
       [-1.51363956,  0.39638424],
       [-0.12577794, -0.26340503],
       [ 0.86586672,  0.41563491]])

In [28]:
y

array([1, 1, 1, ..., 0, 1, 0], dtype=int64)

In [29]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.2, random_state=42)

In [30]:
len(X_test)

2000

In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
tree_clf = DecisionTreeClassifier()
parameters = {'criterion':['gini','entropy'],
              'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40],
              'min_samples_leaf':[1,3,5,10,15,20],
              'max_leaf_nodes':[1,3,4,10,15,20,25,30,35,40,45,50]}
clf =GridSearchCV(tree_clf , parameters, cv=5)
clf.fit(X_train, y_train)

F:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
780 fits failed out of a total of 9360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
780 fits failed with the following error:
Traceback (most recent call last):
  File "F:\Anaconda\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "F:\Anaconda\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "F:\Anaconda\lib\site-packages\sklearn\tree\_classes.py", line 314, in fit
    raise ValueError(
ValueError: max_leaf_nodes 1 must be either None or larger than 1

  warnings.warn(some_fits_failed_message, FitFailedWarning)
F:

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 20, 30,
                                       40],
                         'max_leaf_nodes': [1, 3, 4, 10, 15, 20, 25, 30, 35, 40,
                                            45, 50],
                         'min_samples_leaf': [1, 3, 5, 10, 15, 20]})

In [32]:
from sklearn.metrics import accuracy_score
y_pre = clf.best_estimator_.predict(X)
accuracy_score(y, y_pre)


0.8667

In [33]:
len(X_train)-n_instances

7900

In [34]:
from sklearn.model_selection import ShuffleSplit
n_trees = 1000
n_instances = 100
# 100個のインスタンスをもつ1000個のサブセットを作りたい
ss = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)

In [35]:
mini_sets = []
for mini_train_index, mini_test_index in ss.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [68]:
from sklearn.base import clone

forest = [clone(clf.best_estimator_) for _ in range(n_trees)]
# モデルを1000回複製してリストにする（１０００回のループで使いやすくする）

accuracy_scores = []
for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))
np.mean(accuracy_scores)

0.7748769999999999

In [69]:
Y_pred = [[0]*len(X_test) for i in range(n_trees)]
for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)
    

In [70]:
Y_pred

[array([1, 1, 0, ..., 0, 0, 1], dtype=int64),
 array([1, 1, 0, ..., 0, 0, 1], dtype=int64),
 array([1, 1, 0, ..., 1, 1, 1], dtype=int64),
 array([1, 1, 0, ..., 1, 1, 0], dtype=int64),
 array([0, 0, 0, ..., 1, 1, 1], dtype=int64),
 array([1, 1, 0, ..., 0, 0, 1], dtype=int64),
 array([1, 1, 0, ..., 0, 0, 1], dtype=int64),
 array([1, 1, 0, ..., 1, 1, 1], dtype=int64),
 array([1, 1, 0, ..., 1, 1, 0], dtype=int64),
 array([1, 1, 0, ..., 0, 0, 0], dtype=int64),
 array([1, 1, 0, ..., 1, 1, 1], dtype=int64),
 array([1, 1, 0, ..., 1, 1, 0], dtype=int64),
 array([1, 1, 0, ..., 0, 0, 0], dtype=int64),
 array([1, 1, 0, ..., 1, 1, 0], dtype=int64),
 array([1, 1, 0, ..., 1, 1, 0], dtype=int64),
 array([1, 1, 0, ..., 0, 0, 1], dtype=int64),
 array([1, 1, 0, ..., 1, 1, 0], dtype=int64),
 array([1, 1, 0, ..., 0, 0, 1], dtype=int64),
 array([1, 1, 0, ..., 1, 0, 0], dtype=int64),
 array([1, 1, 0, ..., 1, 0, 0], dtype=int64),
 array([1, 1, 0, ..., 1, 1, 0], dtype=int64),
 array([1, 1, 0, ..., 0, 1, 1], dt

In [71]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [73]:
y_pred_majority_votes

array([[1, 1, 0, ..., 1, 1, 0]], dtype=int64)

In [74]:
n_votes

array([[992, 992, 996, ..., 648, 661, 529]])

In [72]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.8015